In [39]:
from pathlib import Path
import sys
from itertools import chain

import pandas as pd
from sklearn import preprocessing
import numpy as np

current_directory = Path().absolute().parent
udf_directory = current_directory.joinpath('src','baseline_algorithm')
sys.path.append(str(udf_directory)) # appendの際にはstrに変換しておく必要あり
default_data_directory = current_directory.joinpath('data')

In [8]:
def flatten(nestedList):
    return list(chain.from_iterable([[item] if type(item)!=list else flatten(item) for item in nestedList]))

# devide item properties into lists
items_path = default_data_directory.joinpath('item_metadata.csv')
items = pd.read_csv(items_path)
tmp =items['properties'].apply(lambda x: x.split('|'))

# create flat list for label encoding
flat_list = flatten(tmp)
properties_set = list(set(flat_list))
print('num set properties: ',len(properties_set))

In [34]:
# label encoding
# 後で辞書とぶつけて変数名を復活させるほうが早そう
le = preprocessing.LabelEncoder()
le.fit(properties_set)
# 一応中身の確認
sorted_encoded = np.sort(le.transform(properties_set))
print('encoded items: ',sorted_encoded)

trans = [le.transform(i) for i in tmp]


LabelEncoder()

In [54]:
# create binary item properties
emp = np.zeros([len(tmp), len(sorted_encoded)])
for tr in range(len(trans)):
    for t in trans[tr]:
        emp[tr,t]=1

In [66]:
# label encoded
item_props = pd.DataFrame(emp).astype('int')
column_name = le.inverse_transform(sorted_encoded)
item_props.columns = column_name
# item_props.head()

item_props['item_id'] = items['item_id']

# item_props.head()

,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,Airport Hotel,...,Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
0,0,0,0,1,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,1,1
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,1,1
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,1
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,1,0,1,1,0,0,0,...,1,0,1,0,1,0,0,1,1,1


In [ ]:
item_props.to_csv('recsys2019_preprocessed_item_properties.csv')